In [1]:
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

In [28]:
agrid = np.ones(100)*0.01
data = np.random.rand(3*100*100).reshape((100,100, 3))
data[:, 1] *= 2
data[:, 2] *= 3

In [29]:
output_notebook()

Loading BokehJS ...

In [30]:
from bokeh.layouts import column
from bokeh.models import CustomJS, Panel, Tabs
from bokeh.models.widgets import Slider

max_periods = 100

#bar_widths = np.zeros(baselineMod.par.agrid.shape)
#bar_widths[1:] = baselineMod.par.agrid[1:] - baselineMod.par.agrid[:-1]
#bar_widths[0] = bar_widths[1]
#
#D_marg = np.sum(baselineMod.path.D, axis=(1, 3))
#change = np.zeros(D_marg.shape)
#change[1:] = D_marg[1:] - D_marg[:-1]

bar_widths = agrid
change = data

x = agrid

source0 = {'x': x, 'y': change[0, :, 0], 'width': bar_widths}
source0 = ColumnDataSource(data=source0)
source1 = {'x': x, 'y': change[0, :, 1], 'width': bar_widths}
source1 = ColumnDataSource(data=source1)

all_slices0 = {}
all_slices1 = {}
for t in range(max_periods):
    all_slices0[str(t)] = list(change[t, :, 0])
    all_slices1[str(t)] = list(change[t, :, 1])

p0 = figure(plot_width=900, plot_height=400, tools='hover,box_zoom,reset')
p0.vbar(x='x', top='y', width='width', fill_color='navy', line_color='navy', source=source0)
tab0 = Panel(child=p0, title='Sector 0')

p1 = figure(plot_width=900, plot_height=400, tools='hover,box_zoom,reset')
p1.vbar(x='x', top='y', width='width', fill_color='olive', line_color='olive', source=source1)
tab1 = Panel(child=p1, title='Sector 1')

handler = CustomJS(args=dict(source0=source0, source1=source1, all_slices0=all_slices0, all_slices1=all_slices1), code="""
   var data0 = source0.data
   var data1 = source1.data
   var f = cb_obj.value
   var fstr = f.toString()
   data0['y'] = all_slices0[fstr]
   data1['y'] = all_slices1[fstr]
   
   source0.change.emit()
   source1.change.emit()
""")

slider = Slider(start=0, end=max_periods, value=0, step=1, title="Slider Value")

slider.js_on_change('value', handler)
layout = column(slider, Tabs(tabs=[tab0, tab1]))
show(layout)